In [ ]:
import numpy as np 
import pandas as pd
from tqdm import tqdm
from PIL import Image
import random
import os
import warnings
warnings.filterwarnings('ignore')
import lightgbm as lgb
from sklearn.model_selection import GroupKFold, StratifiedKFold, KFold
from sklearn import metrics
from sklearn import preprocessing
from bayes_opt import BayesianOptimization
from catboost import CatBoostClassifier
import gc

In [ ]:

csv1 = ['../input/melanoma-latest-oofs/exp38450_oof_16tta_92511.csv',
        '../input/melanoma-latest-oofs/exp38450_oof_notta_92869.csv',
        '../input/melanoma-latest-oofs/exp38450_oof_oldaug_3tta_91687.csv',
        '../input/melanoma-latest-oofs/exp38470_oof_16tta_92401.csv',
        '../input/melanoma-latest-oofs/exp38470_oof_notta_91759.csv',
        '../input/melanoma-latest-oofs/exp38470_oof_oldaug_3tta_91198.csv',
        '../input/melanoma-latest-oofs/exp3847_oof_16tta_92082.csv',
        '../input/melanoma-latest-oofs/exp3847_oof_notta_90459.csv',
        '../input/melanoma-latest-oofs/exp3847_oof_oldaug_3tta_90717.csv',
        '../input/melanoma-latest-oofs/exp5123_new_oof_16tta_93036_580.csv',
        '../input/melanoma-latest-oofs/exp5123_new_oof_notta_93222_580.csv',
        '../input/melanoma-latest-oofs/exp5123_new_oof_22tta_92431.csv',
        '../input/melanoma-latest-oofs/exp5123_new_oof_notta_92598.csv',
        '../input/melanoma-latest-oofs/exp5123_new_oof_oldaug_3tta_90901.csv',
        '../input/melanoma-latest-oofs/exp5123_oof_16tta_93063.csv',
        '../input/melanoma-latest-oofs/exp5123_oof_notta_92904.csv',
        '../input/melanoma-latest-oofs/exp5123_oof_oldaug_3tta_90996.csv',
        '../input/melanoma-latest-oofs/exp51250_oof_16tta_92383.csv',
        '../input/melanoma-latest-oofs/exp51250_oof_notta_92534.csv',
        '../input/melanoma-latest-oofs/exp51250_oof_oldaug_3tta_91027.csv',
        '../input/melanoma-latest-oofs/exp5125_oof_16tta_93209.csv',
        '../input/melanoma-latest-oofs/exp5125_oof_notta_92885.csv',
        '../input/melanoma-latest-oofs/exp5125_oof_oldaug_3tta_91750.csv',
        '../input/melanoma-latest-oofs/exp51270_oof_16tta_92738.csv',
        '../input/melanoma-latest-oofs/exp51270_oof_notta_92153.csv',
        '../input/melanoma-latest-oofs/exp51270_oof_oldaug_3tta_90757.csv',
        '../input/melanoma-latest-oofs/exp5127_oof_16tta_92403.csv',
        '../input/melanoma-latest-oofs/exp5127_oof_notta_92489.csv',
        '../input/melanoma-latest-oofs/exp5127_oof_oldaug_3tta_91805.csv',
        '../input/melanoma-latest-oofs/exp512dpn131_oof_16tta_92108.csv',
        '../input/melanoma-latest-oofs/exp512dpn131_oof_notta_92274.csv',
        '../input/melanoma-latest-oofs/exp512dpn131_oof_oldaug_3tta_89954.csv',
        '../input/melanoma-latest-oofs/exp512se50_oof_16tta_92062.csv',
        '../input/melanoma-latest-oofs/exp512se50_oof_notta_92788.csv',
        '../input/melanoma-latest-oofs/exp512se50_oof_oldaug_3tta_90352.csv',
        '../input/melanoma-latest-oofs/exp512sk50_oof_16tta_91944.csv',
        '../input/melanoma-latest-oofs/exp512sk50_oof_notta_91991.csv',
        '../input/melanoma-latest-oofs/exp512sk50_oof_oldaug_3tta_90370.csv',
        '../input/melanoma-latest-oofs/exp7680_oof_oldaug_3tta_91116.csv',
        '../input/melanoma-latest-oofs/exp7683_oof_16tta_92352.csv',
        '../input/melanoma-latest-oofs/exp7683_oof_notta_92670.csv',
        '../input/melanoma-latest-oofs/exp7683_oof_oldaug_3tta_92054.csv']

In [ ]:
subs = []
for index, path in enumerate(csv1):
    df = pd.read_csv(path)
    df.columns = ['image_name', f'preds{index}']
    subs.append(df)
train_oof = subs[0]

for sub_index in range(1, len(subs)):
    train_oof = train_oof.merge(subs[sub_index], on='image_name')

In [ ]:
train_oof.head()

In [ ]:
train_oof.shape

In [ ]:
csv = [
    
    
 '../input/melanoma-latest-subs/38450_submission_1_16tta.csv', #0.9251
 '../input/melanoma-latest-subs/38450_submission_2_16tta.csv',
 '../input/melanoma-latest-subs/38450_submission_3_16tta.csv',
 '../input/melanoma-latest-subs/38450_submission_4_16tta.csv',
 '../input/melanoma-latest-subs/38450_submission_5_16tta.csv',
    
        
 '../input/melanoma-latest-subs/38450_submission_1_notta.csv', #0.9286
 '../input/melanoma-latest-subs/38450_submission_2_notta.csv',
 '../input/melanoma-latest-subs/38450_submission_3_notta.csv',
 '../input/melanoma-latest-subs/38450_submission_4_notta.csv',
 '../input/melanoma-latest-subs/38450_submission_5_notta.csv',
    
 '../input/melanoma-latest-subs/38450_oldaug_submission_1_3tta.csv', # 0.9168
 '../input/melanoma-latest-subs/38450_oldaug_submission_2_3tta.csv',
 '../input/melanoma-latest-subs/38450_oldaug_submission_3_3tta.csv',
 '../input/melanoma-latest-subs/38450_oldaug_submission_4_3tta.csv',
 '../input/melanoma-latest-subs/38450_oldaug_submission_5_3tta.csv',
    
 '../input/melanoma-latest-subs/38470_submission_1_16tta.csv', # 0.9240
 '../input/melanoma-latest-subs/38470_submission_2_16tta.csv',
 '../input/melanoma-latest-subs/38470_submission_3_16tta.csv',
 '../input/melanoma-latest-subs/38470_submission_4_16tta.csv',
 '../input/melanoma-latest-subs/38470_submission_5_16tta.csv',
    
 '../input/melanoma-latest-subs/38470_submission_1_notta.csv', # 9175
 '../input/melanoma-latest-subs/38470_submission_2_notta.csv',
 '../input/melanoma-latest-subs/38470_submission_3_notta.csv',
 '../input/melanoma-latest-subs/38470_submission_4_notta.csv',
 '../input/melanoma-latest-subs/38470_submission_5_notta.csv',   

    
    
    
 '../input/melanoma-latest-subs/38470_oldaug_submission_1_3tta.csv', # 0.9119
 '../input/melanoma-latest-subs/38470_oldaug_submission_2_3tta.csv',
 '../input/melanoma-latest-subs/38470_oldaug_submission_3_3tta.csv',
 '../input/melanoma-latest-subs/38470_oldaug_submission_4_3tta.csv',
 '../input/melanoma-latest-subs/38470_oldaug_submission_5_3tta.csv',    
    

 '../input/melanoma-latest-subs/3847_submission_1_16tta.csv', # 0.9208
 '../input/melanoma-latest-subs/3847_submission_2_16tta.csv',
 '../input/melanoma-latest-subs/3847_submission_3_16tta.csv',
 '../input/melanoma-latest-subs/3847_submission_4_16tta.csv',
 '../input/melanoma-latest-subs/3847_submission_5_16tta.csv',
        
 '../input/melanoma-latest-subs/3847_submission_1_notta.csv', # 9045
 '../input/melanoma-latest-subs/3847_submission_2_notta.csv',
 '../input/melanoma-latest-subs/3847_submission_3_notta.csv',
 '../input/melanoma-latest-subs/3847_submission_4_notta.csv',
 '../input/melanoma-latest-subs/3847_submission_5_notta.csv',    

    
 '../input/melanoma-latest-subs/3847_oldaug_submission_1_3tta.csv', # 0.9071
 '../input/melanoma-latest-subs/3847_oldaug_submission_2_3tta.csv',
 '../input/melanoma-latest-subs/3847_oldaug_submission_3_3tta.csv',
 '../input/melanoma-latest-subs/3847_oldaug_submission_4_3tta.csv',
 '../input/melanoma-latest-subs/3847_oldaug_submission_5_3tta.csv',      
    
    
 '../input/melanoma-latest-subs/5123_new_submission_1_16tta_580.csv', # 0.930
 '../input/melanoma-latest-subs/5123_new_submission_2_16tta_580.csv',
 '../input/melanoma-latest-subs/5123_new_submission_3_16tta_580.csv',
 '../input/melanoma-latest-subs/5123_new_submission_4_16tta_580.csv',
 '../input/melanoma-latest-subs/5123_new_submission_5_16tta_580.csv',
    
    
 '../input/melanoma-latest-subs/5123_new_submission_1_notta_580.csv', # 0.932
 '../input/melanoma-latest-subs/5123_new_submission_2_notta_580.csv',
 '../input/melanoma-latest-subs/5123_new_submission_3_notta_580.csv',
 '../input/melanoma-latest-subs/5123_new_submission_4_notta_580.csv',
 '../input/melanoma-latest-subs/5123_new_submission_5_notta_580.csv',
    
    
    
    
    
    
 '../input/melanoma-latest-subs/5123_new_submission_1_22tta.csv', # 0.924
 '../input/melanoma-latest-subs/5123_new_submission_2_22tta.csv',
 '../input/melanoma-latest-subs/5123_new_submission_3_22tta.csv',
 '../input/melanoma-latest-subs/5123_new_submission_4_22tta.csv',
 '../input/melanoma-latest-subs/5123_new_submission_5_22tta.csv',
    
 '../input/melanoma-latest-subs/5123_new_submission_1_notta.csv', # 925
 '../input/melanoma-latest-subs/5123_new_submission_2_notta.csv',
 '../input/melanoma-latest-subs/5123_new_submission_3_notta.csv',
 '../input/melanoma-latest-subs/5123_new_submission_4_notta.csv',
 '../input/melanoma-latest-subs/5123_new_submission_5_notta.csv', 
    
    
 '../input/melanoma-latest-subs/5123_new_oldaug_submission_1_3tta.csv', # 0.909
 '../input/melanoma-latest-subs/5123_new_oldaug_submission_2_3tta.csv',
 '../input/melanoma-latest-subs/5123_new_oldaug_submission_3_3tta.csv',
 '../input/melanoma-latest-subs/5123_new_oldaug_submission_4_3tta.csv',
 '../input/melanoma-latest-subs/5123_new_oldaug_submission_5_3tta.csv', 
    
    
 '../input/melanoma-latest-subs/5123_submission_1_16tta.csv', # 0.930
 '../input/melanoma-latest-subs/5123_submission_2_16tta.csv',
 '../input/melanoma-latest-subs/5123_submission_3_16tta.csv',
 '../input/melanoma-latest-subs/5123_submission_4_16tta.csv',
 '../input/melanoma-latest-subs/5123_submission_5_16tta.csv',
        
 '../input/melanoma-latest-subs/5123_submission_1_notta.csv', # 929
 '../input/melanoma-latest-subs/5123_submission_2_notta.csv',
 '../input/melanoma-latest-subs/5123_submission_3_notta.csv',
 '../input/melanoma-latest-subs/5123_submission_4_notta.csv',
 '../input/melanoma-latest-subs/5123_submission_5_notta.csv',    

 '../input/melanoma-latest-subs/5123_oldaug_submission_1_3tta.csv', # 0.909
 '../input/melanoma-latest-subs/5123_oldaug_submission_2_3tta.csv',
 '../input/melanoma-latest-subs/5123_oldaug_submission_3_3tta.csv',
 '../input/melanoma-latest-subs/5123_oldaug_submission_4_3tta.csv',
 '../input/melanoma-latest-subs/5123_oldaug_submission_5_3tta.csv', 
    
    
    
 '../input/melanoma-latest-subs/51250_submission_1_16tta.csv', # 0.923
 '../input/melanoma-latest-subs/51250_submission_2_16tta.csv',
 '../input/melanoma-latest-subs/51250_submission_3_16tta.csv',
 '../input/melanoma-latest-subs/51250_submission_4_16tta.csv',
 '../input/melanoma-latest-subs/51250_submission_5_16tta.csv',
        
 '../input/melanoma-latest-subs/51250_submission_1_notta.csv', # 925
 '../input/melanoma-latest-subs/51250_submission_2_notta.csv',
 '../input/melanoma-latest-subs/51250_submission_3_notta.csv',
 '../input/melanoma-latest-subs/51250_submission_4_notta.csv',
 '../input/melanoma-latest-subs/51250_submission_5_notta.csv',    

 '../input/melanoma-latest-subs/51250_oldaug_submission_1_3tta.csv', # 0.910
 '../input/melanoma-latest-subs/51250_oldaug_submission_2_3tta.csv',
 '../input/melanoma-latest-subs/51250_oldaug_submission_3_3tta.csv',
 '../input/melanoma-latest-subs/51250_oldaug_submission_4_3tta.csv',
 '../input/melanoma-latest-subs/51250_oldaug_submission_5_3tta.csv', 
    
 
 '../input/melanoma-latest-subs/5125_submission_1_16tta.csv', # 0.932
 '../input/melanoma-latest-subs/5125_submission_2_16tta.csv',
 '../input/melanoma-latest-subs/5125_submission_3_16tta.csv',
 '../input/melanoma-latest-subs/5125_submission_4_16tta.csv',
 '../input/melanoma-latest-subs/5125_submission_5_16tta.csv',
        
 '../input/melanoma-latest-subs/5125_submission_1_notta.csv', # 928
 '../input/melanoma-latest-subs/5125_submission_2_notta.csv',
 '../input/melanoma-latest-subs/5125_submission_3_notta.csv',
 '../input/melanoma-latest-subs/5125_submission_4_notta.csv',
 '../input/melanoma-latest-subs/5125_submission_5_notta.csv',    

 '../input/melanoma-latest-subs/5125_oldaug_submission_1_3tta.csv', # 0.917
 '../input/melanoma-latest-subs/5125_oldaug_submission_2_3tta.csv',
 '../input/melanoma-latest-subs/5125_oldaug_submission_3_3tta.csv',
 '../input/melanoma-latest-subs/5125_oldaug_submission_4_3tta.csv',
 '../input/melanoma-latest-subs/5125_oldaug_submission_5_3tta.csv', 
    

    
    
 '../input/melanoma-latest-subs/51270_submission_1_16tta.csv', # 0.927
 '../input/melanoma-latest-subs/51270_submission_2_16tta.csv',
 '../input/melanoma-latest-subs/51270_submission_3_16tta.csv',
 '../input/melanoma-latest-subs/51270_submission_4_16tta.csv',
 '../input/melanoma-latest-subs/51270_submission_5_16tta.csv',
        
 '../input/melanoma-latest-subs/51270_submission_1_notta.csv', # .921
 '../input/melanoma-latest-subs/51270_submission_2_notta.csv',
 '../input/melanoma-latest-subs/51270_submission_3_notta.csv',
 '../input/melanoma-latest-subs/51270_submission_4_notta.csv',
 '../input/melanoma-latest-subs/51270_submission_5_notta.csv',    

 '../input/melanoma-latest-subs/51270_oldaug_submission_1_3tta.csv', # 0.907
 '../input/melanoma-latest-subs/51270_oldaug_submission_2_3tta.csv',
 '../input/melanoma-latest-subs/51270_oldaug_submission_3_3tta.csv',
 '../input/melanoma-latest-subs/51270_oldaug_submission_4_3tta.csv',
 '../input/melanoma-latest-subs/51270_oldaug_submission_5_3tta.csv', 
    
    
 '../input/melanoma-latest-subs/5127_submission_1_16tta.csv', # 0.924
 '../input/melanoma-latest-subs/5127_submission_2_16tta.csv',
 '../input/melanoma-latest-subs/5127_submission_3_16tta.csv',
 '../input/melanoma-latest-subs/5127_submission_4_16tta.csv',
 '../input/melanoma-latest-subs/5127_submission_5_16tta.csv',
        
 '../input/melanoma-latest-subs/5127_submission_1_notta.csv', # 924
 '../input/melanoma-latest-subs/5127_submission_2_notta.csv',
 '../input/melanoma-latest-subs/5127_submission_3_notta.csv',
 '../input/melanoma-latest-subs/5127_submission_4_notta.csv',
 '../input/melanoma-latest-subs/5127_submission_5_notta.csv',    

 '../input/melanoma-latest-subs/5127_oldaug_submission_1_3tta.csv', # 0.918
 '../input/melanoma-latest-subs/5127_oldaug_submission_2_3tta.csv',
 '../input/melanoma-latest-subs/5127_oldaug_submission_3_3tta.csv',
 '../input/melanoma-latest-subs/5127_oldaug_submission_4_3tta.csv',
 '../input/melanoma-latest-subs/5127_oldaug_submission_5_3tta.csv', 
    
    
    
    
 '../input/melanoma-latest-subs/512dpn131_submission_1_16tta.csv', # 0.921
 '../input/melanoma-latest-subs/512dpn131_submission_2_16tta.csv',
 '../input/melanoma-latest-subs/512dpn131_submission_3_16tta.csv',
 '../input/melanoma-latest-subs/512dpn131_submission_4_16tta.csv',
 '../input/melanoma-latest-subs/512dpn131_submission_5_16tta.csv',
        
 '../input/melanoma-latest-subs/512dpn131_submission_1_notta.csv', # 922
 '../input/melanoma-latest-subs/512dpn131_submission_2_notta.csv',
 '../input/melanoma-latest-subs/512dpn131_submission_3_notta.csv',
 '../input/melanoma-latest-subs/512dpn131_submission_4_notta.csv',
 '../input/melanoma-latest-subs/512dpn131_submission_5_notta.csv',    

    
 '../input/melanoma-latest-subs/512dpn131_oldaug_submission_1_3tta.csv', # 0.899
 '../input/melanoma-latest-subs/512dpn131_oldaug_submission_2_3tta.csv',
 '../input/melanoma-latest-subs/512dpn131_oldaug_submission_3_3tta.csv',
 '../input/melanoma-latest-subs/512dpn131_oldaug_submission_4_3tta.csv',
 '../input/melanoma-latest-subs/512dpn131_oldaug_submission_5_3tta.csv', 
    

 '../input/melanoma-latest-subs/512se50_submission_1_16tta.csv', # 0.920
 '../input/melanoma-latest-subs/512se50_submission_2_16tta.csv',
 '../input/melanoma-latest-subs/512se50_submission_3_16tta.csv',
 '../input/melanoma-latest-subs/512se50_submission_4_16tta.csv',
 '../input/melanoma-latest-subs/512se50_submission_5_16tta.csv',
        
 '../input/melanoma-latest-subs/512se50_submission_1_notta.csv', # 927
 '../input/melanoma-latest-subs/512se50_submission_2_notta.csv',
 '../input/melanoma-latest-subs/512se50_submission_3_notta.csv',
 '../input/melanoma-latest-subs/512se50_submission_4_notta.csv',
 '../input/melanoma-latest-subs/512se50_submission_5_notta.csv',    

    
    
 '../input/melanoma-latest-subs/512se50_oldaug_submission_1_3tta.csv', # 0.903
 '../input/melanoma-latest-subs/512se50_oldaug_submission_2_3tta.csv',
 '../input/melanoma-latest-subs/512se50_oldaug_submission_3_3tta.csv',
 '../input/melanoma-latest-subs/512se50_oldaug_submission_4_3tta.csv',
 '../input/melanoma-latest-subs/512se50_oldaug_submission_5_3tta.csv', 
    
    
    
 '../input/melanoma-latest-subs/512sk50_submission_1_16tta.csv', # 0.919
 '../input/melanoma-latest-subs/512sk50_submission_2_16tta.csv',
 '../input/melanoma-latest-subs/512sk50_submission_3_16tta.csv',
 '../input/melanoma-latest-subs/512sk50_submission_4_16tta.csv',
 '../input/melanoma-latest-subs/512sk50_submission_5_16tta.csv',
        
 '../input/melanoma-latest-subs/512sk50_submission_1_notta.csv', # 919
 '../input/melanoma-latest-subs/512sk50_submission_2_notta.csv',
 '../input/melanoma-latest-subs/512sk50_submission_3_notta.csv',
 '../input/melanoma-latest-subs/512sk50_submission_4_notta.csv',
 '../input/melanoma-latest-subs/512sk50_submission_5_notta.csv',    

    
 '../input/melanoma-latest-subs/512sk50_oldaug_submission_1_3tta.csv', # 0.903
 '../input/melanoma-latest-subs/512sk50_oldaug_submission_2_3tta.csv',
 '../input/melanoma-latest-subs/512sk50_oldaug_submission_3_3tta.csv',
 '../input/melanoma-latest-subs/512sk50_oldaug_submission_4_3tta.csv',
 '../input/melanoma-latest-subs/512sk50_oldaug_submission_5_3tta.csv', 
    
    
  
 '../input/melanoma-latest-subs/7680_oldaug_submission_1_3tta.csv', # 0.9116
 '../input/melanoma-latest-subs/7680_oldaug_submission_2_3tta.csv',
 '../input/melanoma-latest-subs/7680_oldaug_submission_3_3tta.csv',
 '../input/melanoma-latest-subs/7680_oldaug_submission_4_3tta.csv',
 '../input/melanoma-latest-subs/7680_oldaug_submission_5_3tta.csv', 
    

 '../input/melanoma-latest-subs/7683_submission_1_16tta.csv', # 0.923
 '../input/melanoma-latest-subs/7683_submission_2_16tta.csv',
 '../input/melanoma-latest-subs/7683_submission_3_16tta.csv',
 '../input/melanoma-latest-subs/7683_submission_4_16tta.csv',
 '../input/melanoma-latest-subs/7683_submission_5_16tta.csv',
        
 '../input/melanoma-latest-subs/7683_submission_1_notta.csv', # 926
 '../input/melanoma-latest-subs/7683_submission_2_notta.csv',
 '../input/melanoma-latest-subs/7683_submission_3_notta.csv',
 '../input/melanoma-latest-subs/7683_submission_4_notta.csv',
 '../input/melanoma-latest-subs/7683_submission_5_notta.csv',    

    
 '../input/melanoma-latest-subs/7683_oldaug_submission_1_3tta.csv', # 0.920
 '../input/melanoma-latest-subs/7683_oldaug_submission_2_3tta.csv',
 '../input/melanoma-latest-subs/7683_oldaug_submission_3_3tta.csv',
 '../input/melanoma-latest-subs/7683_oldaug_submission_4_3tta.csv',
 '../input/melanoma-latest-subs/7683_oldaug_submission_5_3tta.csv', 
    
    
]

In [ ]:
subs = []
for index, path in enumerate(csv):
    df = pd.read_csv(path)
    df.columns = ['image_name', f'target{index}']
    subs.append(df)

test_oof = subs[0]

for sub_index in range(1, len(subs)):
    test_oof = test_oof.merge(subs[sub_index], on='image_name')
for i in range(42):
    test_oof['preds'+str(i)] = test_oof[list(test_oof.columns.values[i*5:(i+1)*5])].mean(axis=1)

In [ ]:
test_oof.drop(test_oof.columns[[x for x in range(1,211)]], axis=1, inplace=True)

In [ ]:
test_oof.head()

In [ ]:
test_oof.shape

In [ ]:
## function to seed everything
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
# define a seed
SEED = 42
    
# let´s start seeding everything
seed_everything(SEED)

# function to read data and image data models predictions
def read_data():
    train = pd.read_csv('../input/melanoma-csv/train_with_mean_rgb.csv')
    test = pd.read_csv('../input/melanoma-csv/test_with_mean_rgb.csv')
    sub = pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/sample_submission.csv')
    train1 = pd.read_csv('../input/oofs-melanoma/oof_DEDED_9419.csv')
    train2 = pd.read_csv('../input/oofs-melanoma/oof_DenseNet201_9481.csv')
    train3 = pd.read_csv('../input/oofs-melanoma/oof_EB6_9482.csv')
    train4 = pd.read_csv('../input/oofs-melanoma/oof_InceptionResNetV2_9328.csv')

    test1 = pd.read_csv('../input/oofs-melanoma/submission_DEDED_9419.csv')
    test2 = pd.read_csv('../input/oofs-melanoma/submission_DenseNet201_9481.csv')
    test3 = pd.read_csv('../input/oofs-melanoma/submission_EB6_9482.csv')
    test4 = pd.read_csv('../input/oofs-melanoma/submission_InceptionResNetV2_9328.csv')

    
    def fix_predictions(train, test, model):
        test.columns = ['image_name', 'preds{}'.format(model)]
        train = train[['image_name', 'pred']]
        train.columns = ['image_name', 'preds{}'.format(model)]
        return train, test
    
    train1, test1 = fix_predictions(train1, test1, '42')
    train2, test2 = fix_predictions(train2, test2, '43')
    train3, test3 = fix_predictions(train3, test3, '44')
    train4, test4 = fix_predictions(train4, test4, '45')


    train = train.merge(train1, on = 'image_name').merge(train2, on = 'image_name').merge(train3, on = 'image_name').merge(train4, on = 'image_name')
    test = test.merge(test1, on = 'image_name').merge(test2, on = 'image_name').merge(test3, on = 'image_name').merge(test4, on = 'image_name')
    return train, test, sub

train, test, sub = read_data()

In [ ]:
train.shape

In [ ]:
train.head()

In [ ]:
train = train.merge(train_oof, on = 'image_name')
test = test.merge(test_oof, on = 'image_name')

In [ ]:
train.columns

In [ ]:
def print_roc_auc(df):
    for i in range(46):
        roc_auc = metrics.roc_auc_score(df['target'], df['preds'+str(i)])
        print(f'Our model {i} out of folds roc auc score is {roc_auc}')
        print('-'*50)
        print('\n')
        
print_roc_auc(train)

In [ ]:
train.head()

In [ ]:
test.columns

In [ ]:
def feature_engineering(train, test):
    # size of images
    trn_images = train['image_name'].values
    trn_sizes = np.zeros((trn_images.shape[0], 2))
    for i, img_path in enumerate(tqdm(trn_images)):
        img = Image.open(os.path.join('/kaggle/input/siim-isic-melanoma-classification/jpeg/train/', f'{img_path}.jpg'))
        trn_sizes[i] = np.array([img.size[0], img.size[1]])
    test_images = test['image_name'].values
    test_sizes = np.zeros((test_images.shape[0],2))
    for i, img_path in enumerate(tqdm(test_images)):
        img = Image.open(os.path.join('/kaggle/input/siim-isic-melanoma-classification/jpeg/test/', f'{img_path}.jpg'))
        test_sizes[i] = np.array([img.size[0],img.size[1]])
    train['w'] = trn_sizes[:,0]
    train['h'] = trn_sizes[:,1]
    test['w'] = test_sizes[:,0]
    test['h'] = test_sizes[:,1]
    
    return train, test

train, test = feature_engineering(train, test)

In [ ]:
train_bk = train
test_bk = test

In [ ]:
train = train_bk
test = test_bk

In [ ]:
train.shape

In [ ]:
#train['group'] = train['patient_id'].apply(lambda x:x[4]).astype(int)
train['group2'] = train['patient_id'].apply(lambda x:x[4]+x[5]+x[6]+x[7]+x[8]+x[9]).astype(int)

#test['group'] = test['patient_id'].apply(lambda x:x[9]).astype(int)
#test['group2'] = test['patient_id'].apply(lambda x:x[9]).astype(int)

test['group2'] = test['patient_id'].apply(lambda x:x[4]+x[5]+x[6]+x[7]+x[8]+x[9]).astype(int)


In [ ]:
#train_temp = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')
#test_temp = pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')
#train['sex'] = train_temp['sex']
#test['sex'] = test_temp['sex']
#train['anatom_site_general_challenge'] = train_temp['anatom_site_general_challenge']
#test['anatom_site_general_challenge'] = test_temp['anatom_site_general_challenge']

def encode_categorical(train, test):
    for col in ['sex', 'anatom_site_general_challenge']:
        encoder = preprocessing.LabelEncoder()
        train[col].fillna('unknown', inplace = True)
        test[col].fillna('unknown', inplace = True)
        train[col] = encoder.fit_transform(train[col])
        test[col] = encoder.transform(test[col])
    age_approx = np.nanmean(np.concatenate([np.array(train['age_approx']), np.array(test['age_approx'])]))
    train['age_approx'].fillna(age_approx, inplace = True)
    test['age_approx'].fillna(age_approx, inplace = True)
    train['patient_id'].fillna('unknown', inplace = True)
    return train, test


train, test = encode_categorical(train, test)

In [ ]:
dups=pd.read_csv('/kaggle/input/siim-list-of-duplicates/2020_Challenge_duplicates.csv')
#train_duplicates = dups[dups['partition']=='train']['ISIC_id']
#train = train[~train['image_name'].isin(train_duplicates)].reset_index(drop=True)

In [ ]:
train.shape

In [ ]:
def train_and_evaluate_lgbm(train, test, params, verbose_eval, folds = 5):
    
    # define usefull features
    features = [col for col in train.columns if col not in ['image_name', 'patient_id', 'diagnosis', 'benign_malignant', 'target', 'source', 'split', 'tfrecord']]
    if verbose_eval != False:
        print('Training with features: ', features)
    
    
    # groupkfolds to predict evaluate unknown clients (just like the test set)
    kf = GroupKFold(n_splits = folds)
    target = 'target'
    
    oof_pred = np.zeros(len(train))
    y_pred = np.zeros(len(test))
     
    for fold, (tr_ind, val_ind) in enumerate(kf.split(train, groups = train['patient_id'])):
        if verbose_eval != False:
            print('\n')
            print('-'*50)
            print(f'Training fold {fold + 1}"')
        x_train, x_val = train[features].iloc[tr_ind], train[features].iloc[val_ind]
        y_train, y_val = train[target][tr_ind], train[target][val_ind]
        train_set = lgb.Dataset(x_train, y_train)
        val_set = lgb.Dataset(x_val, y_val)
        
        model = lgb.train(params, train_set, num_boost_round = 10000, early_stopping_rounds = 50, 
                         valid_sets = [train_set, val_set], verbose_eval = verbose_eval)
        
        
        oof_pred[val_ind] = model.predict(x_val)
        
        y_pred += model.predict(test[features]) / kf.n_splits
        
    rauc = metrics.roc_auc_score(train['target'], oof_pred)
    if verbose_eval != False:
        print(f'Our oof roc auc score for our lgbm model is {rauc}')
        
    gc.collect()
    
    return rauc, y_pred

In [ ]:
def run_lgb_bayesian(num_leaves, learning_rate, max_depth, lambda_l1, lambda_l2, bagging_fraction, bagging_freq, colsample_bytree, colsample_bynode, min_data_per_leaf, min_sum_hessian_per_leaf):
    
    params = {
        'boosting_type': 'gbdt',
        'metric': 'auc',
        'objective': 'binary',
        'n_jobs': -1,
        'seed': SEED,
        'num_leaves': int(num_leaves),
        'learning_rate': learning_rate,
        'max_depth': int(max_depth),
        'lambda_l1': lambda_l1,
        'lambda_l2': lambda_l2,
        'bagging_fraction': bagging_fraction,
        'bagging_freq': int(bagging_freq),
        'colsample_bytree': colsample_bytree,
        'colsample_bynode': colsample_bynode,
        'min_data_per_leaf': int(min_data_per_leaf),
        'min_sum_hessian_per_leaf': min_sum_hessian_per_leaf,
        'verbose': 0
    }
    
    rauc, y_pred = train_and_evaluate_lgbm(train, test, params, False)
    return rauc


# run bayesian optimization with optimal features
bounds_lgb = {
    'num_leaves': (20, 500),
    'learning_rate': (0.01, 0.2),
    'max_depth': (8, 250),
    'lambda_l1': (0, 3),
    'lambda_l2': (0, 3),
    'bagging_fraction': (0.4, 1),
    'bagging_freq': (1, 10),
    'colsample_bytree': (0.4, 1),
    'colsample_bynode': (0.4, 1),
    'min_data_per_leaf': (10, 100),
    'min_sum_hessian_per_leaf': (0.0001, 0.01)
}

lgb_bo = BayesianOptimization(run_lgb_bayesian, bounds_lgb, random_state = SEED)
lgb_bo.maximize(init_points = 300, n_iter = 300, acq = 'ucb', xi = 0.0, alpha = 1e-6)

params = {
    'boosting_type': 'gbdt',
    'metric': 'auc',
    'objective': 'binary',
    'n_jobs': -1,
    'seed': SEED,
    'num_leaves': int(lgb_bo.max['params']['num_leaves']),
    'learning_rate': lgb_bo.max['params']['learning_rate'],
    'max_depth': int(lgb_bo.max['params']['max_depth']),
    'lambda_l1': lgb_bo.max['params']['lambda_l1'],
    'lambda_l2': lgb_bo.max['params']['lambda_l2'],
    'bagging_fraction': lgb_bo.max['params']['bagging_fraction'],
    'bagging_freq': int(lgb_bo.max['params']['bagging_freq']),
    'colsample_bytree': lgb_bo.max['params']['colsample_bytree'],
    'colsample_bynode': lgb_bo.max['params']['colsample_bynode'],
    'min_data_per_leaf': int(lgb_bo.max['params']['min_data_per_leaf']),
    'min_sum_hessian_per_leaf': lgb_bo.max['params']['min_sum_hessian_per_leaf']
}


# train with new hyperparameters
roc_auc, y_pred = train_and_evaluate_lgbm(train, test, params, 50)

# predict
test['target'] = y_pred
sub = test[['image_name', 'target']]
sub.to_csv('lgbm_baseline_sub.csv', index = False)